# Data Load

In [1]:
import numpy as np
import pandas as pd

import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
import warnings
warnings.filterwarnings('ignore')
from matplotlib import font_manager, rc
import scipy as sp
from scipy import special, optimize

In [2]:
font_path="C:/Windows/Fonts/H2HDRM.TTF"
font=font_manager.FontProperties(fname=font_path).get_name()
rc('font',family=font)

In [3]:
df=pd.read_csv("C:/Users/sohee/Desktop/KW/산학연계(졸작)/산학졸작_openUP_Data/kwproja_data_.csv",encoding='utf-8')

# EDA 

153395 rows X 31 columns

- 매장 속성 정보

    - shop_code -> 식별자 feature, input feature로는 사용하지 않지만 분류를 위해서는 사용할 수 있을 것 같음
    - shop_name -> input feature로는 사용하지 않음 (NLP deep learning 가능성 있음)
    - longtitude : 경도, latitude : 위도 -> 매장 위치 (회사 근처, 학교 근처 등 매출 영향성 있음) -> 군집화, labeling 필요
    - shop_type_big -> 13 category -> 업종 (매출 영향성 있음)
    - shop_type_small -> 367 category
    
    
- 매출 정보

    - date -> 12 category, 201702~ 201801 까지의 data
    - monthly_gain / avearge_sale_price = 한달 총 판매수
    
    
- 매출 통계 정보

    - weekday0~6 : 일 ~ 월
    - time -> 05_10 / 10_14 / 14_18 / 18_20 / 20_22 / 22_24 / 24_05 => 시간대가 동일하지 않음
        - 새벽대는 찾는 손님 적어 시간 길게 잡았을 것
        - 저녁 시간대는 가장 매출이 많이 이뤄지는 prime time 이라 시간대를 짧게 잡았을 것
    - female/male : 20/30/40/50 -> 8 columns

monthly_gain과 average_sale_price 중 어느 것을 y값으로 둘 것인가?
- 월매출 예측 문제로 가정하고 montly_gain 을 y값으로 예측하는 모델 만들기

shop_code는 input feature에 넣어야 하는가?
- 특별한 브랜드가 y값을 결정하는 과적합 요소가 될 수 있으므로
- X 에서 shop code, shop name 제외하는 것도 방법
- 어느 위치에 어떤 업종으로 어떤 객단가인 매장을 오픈하면 월매출이 어떻게 될까? 문제
    - X: shop type big, shop type small, longitude, latitude, avg_sale_price,
    - y: montly_gain

In [ ]:
df.head()

In [ ]:
df.columns

In [ ]:
df.info()

## shop_name, shop_code

- shop_code가 식별자 feature, 가게는 동일하지만 가게명이 바뀌는 경우가 존재함
    - shop_name(13,633), shop_code(13,352) -> 약 300 가게 정도는 이름이 바뀐 것으로 추정
    - 확인 결과 총 222가게가 이름을 최소 2번 바꾸었으며 평균 4번, 최대 66번 바꾸었음
- 결론-> shop code, shop name 은 모델입력으로 적합하지 않음
- 업종 별 매출 예측으로만 봐야 적당할 것으로 보임

In [ ]:
# 13633 
# 13352
# shop_name과 shop_code가 항상 같다면, 위 두 숫자가 같아야 하는데 약 300개의 data가 차이가 남 
# shop_code는 동일하지만(가게는 동일하지만) 업종, 가게명이 바뀌었을 수도 있음 -> shop_code를 식별자 feature로 사용
print(df['shop_name'].nunique())
print(df['shop_code'].nunique())
print(df['longitude'].nunique())
print(df['latitude'].nunique())
print(df['shop_type_big'].nunique())
print(df['shop_type_big'].unique())
print(df['shop_type_small'].nunique())
#print(df['shop_type_small'].unique())
print(df['date'].unique())

In [ ]:
# shop code 를 group by 했을 때 그 안에 몇 종류의 shop name이 있는지 (1이어야 정상)
check_df = df.groupby('shop_code')['shop_name'].nunique().to_frame('shop_name_unique').reset_index()
check_df

In [ ]:
# shop code안에 여러 shop name 있는 경우
check_df2 = check_df[check_df['shop_name_unique'] > 1]
check_df2

In [ ]:
# 총 222가게가 이름을 최소 2번 바꾸었으며 평균 4번, 최대 66번 바꾸었음 
check_df2['shop_name_unique'].describe()

In [5]:
# 롯데 월드 안에  놀이기구도 있지만, 편의점, 호텔 등 다는 업종 매출로 잡히는 것들도 있음!
# 결론-> shop code, shop name 은 모델입력으로 적합하지 않음
# 업종 별 매출 예측으로만 봐야 적당할 것으로 보임
df[df['shop_code'] == 148500219]

## shop_type_big, shop_type_small, 업종, 매출 상관관계

- 매출 데이터(monthly_gain)에 missing value 결측치 존재(3,605개) -> 제거하고 확인 총 149,789개
- monthly_gain
    - mean | 98,313,252.03 | 9천8백만
    - min | 5.01 |
    - 25% | 4,443,902.50 | 4백만
    - 50% | 15,570,070.00 | 1천5백만
    - 75% | 43,280,488.20 | 4천3백만
    - max | 110,795,923,219.85 | 1107억9천5백만
- shop type big (13)
    - 음식(51004) 소매(47318) 의료(16618) 생활서비스(16074) 학문/교육(4495) 관광/여가/오락(3676) 도매/유통/무역(2607) 스포츠(1525) 숙박(1198) 제조(518) 부동산(335) 전자/정보통신(98) 기술서비스(21)
    - min
    - '기술서비스', '전자/정보통신', '제조', '의료', '부동산', '숙박', '스포츠', '도매/유통/무역', '음식', '학문/교육', '관광/여가/오락', '소매', '생활서비스'
    - 부동산(10,060) 기준 이하(7곳)은 비슷함
    - mean
    - '도매/유통/무역', '숙박', '제조', '전자/정보통신', '소매', '의료', '관광/여가/오락', '학문/교육', '음식', '생활서비스', '스포츠', '부동산', '기술서비스'
    - max
    - '도매/유통/무역', '소매', '관광/여가/오락', '제조', '숙박', '의료', '전자/정보통신', '생활서비스', '학문/교육', '음식', '스포츠', '부동산', '기술서비스'
- 평균만 놓고 보아도 가장 낮은 업종은 천만원대, 가장 높은 업종은 억대이다
- shop type small (367)

In [ ]:
# issue -> shop_type_big에 관한 issue를 참고하세요!
# issue -> '음식'과 '음식 ' 두 개로 count 되고 있습니다. 아래 코드로 수정하시길 바랍니다

print(df['shop_type_big'].nunique())
print(df['shop_type_big'].unique())

In [ ]:
# 데이터 값 실수. 소수점 두째자리까지 표시
pd.options.display.float_format = '{:.2f}'.format

# 매출데이터에 0값이 3605개가 잡힘 -> 정확한 eda를 위해서 이를 제거하고 계속 진행하겠음 
df_check = df[df.average_sale_price!=0][["shop_type_big", "shop_type_small", "average_sale_price"]].reset_index().drop("index", axis=1)
df_check
#print(153394-3605)

In [ ]:
pd.DataFrame(df_check['average_sale_price']).describe()

In [ ]:
group_big = df_check.groupby('shop_type_big')
df_group_big = group_big.describe().droplevel(axis=1,level=0)
df_group_big

In [ ]:
print("내림차순 기준 분야별로 정렬하여 출력합니다")
print("min: ", list(df_group_big.sort_values(by=['min'], ascending=False).index))
print("mean: ", list(df_group_big.sort_values(by=['mean'], ascending=False).index))
print("max: ", list(df_group_big.sort_values(by=['max'], ascending=False).index))

In [ ]:
# sns.boxplot(x='shop_type_big', y='average_sale_price', data=df_check)

In [ ]:
# group_big.boxplot()

In [ ]:
#fig, axes = plt.subplots(1, 3)

#df_group_big.plot.bar(ax = axes[0], y='min')
#df_group_big.plot.bar(ax = axes[1], y='mean')
#df_group_big.plot.bar(ax = axes[2], y='max')

#plt.rcParams['figure.figsize'] = [14, 6]
#plt.show()

In [ ]:
# density plot only
#ax = plt.subplots()
#ax = sns.distplot(df_check['average_sale_price'], hist = False)
#ax.set_title('average_sale_price Histogram with Density Plot')

# Preprocessing

- shop_code, shop_name : 식별자 feature 이므로 drop
- shop_type_big, shop_type_small : label encodding
- longitude, latitude : 일단은 input_feature에 넣지만 중복값이 많아 보이므로 추후에 제거해보려 함
- monthly_gain : 결측치 제거 (0값, 3605개로 계산됨)
- MinMaxSaclar 정규화 -> 정규화 column의 범위는??
- date : drop, (그러나 RNN, LSTM과 같은 DL 모델에서는 넣어야 할지도..? 시계열 데이터이므로 쓸 수도 없을지도)

In [ ]:
# 원본 data와 따로 관리 -> original data = data, input data = input_data 
# feature drop
input_data = df.copy()

input_data = input_data.drop(['date', 'shop_code', 'shop_name'], axis=1)
input_data

### shop_type labeling

In [ ]:
# LabelEncoder
from sklearn.preprocessing import LabelEncoder

lencoder = LabelEncoder()

# item_big = input_data['shop_type_big'].unique()
# lencoder.fit(item_big)
# input_data['shop_type_big'] = lencoder.transform(input_data['shop_type_big'])

input_data['shop_type_big'] = lencoder.fit_transform(list(input_data['shop_type_big']))

# item_small = input_data['shop_type_small'].unique() 
# lencoder.fit(item_small)
# input_data['shop_type_small'] = lencoder.transform(input_data['shop_type_small'])

input_data['shop_type_small'] = lencoder.fit_transform(list(input_data['shop_type_small']))

input_data

### add geo

In [ ]:
input_data.plot.scatter(x='longitude',y='latitude',grid=True)

In [ ]:
input_data_1=input_data.copy()

In [ ]:
plt.figure(figsize=(15,5))
sns.scatterplot(x='longitude',y='latitude',data=input_data_1)

In [ ]:
# k=9 ?

In [ ]:
centroids = input_data_1[['longitude','latitude']].sample(9, random_state=1)
centroids

In [ ]:
geo_df=input_data_1[['longitude','latitude']]
geo_df

In [ ]:
# 각 데이터에 대하여, 각 중심점과의 유클리드 거리 계산
distance = sp.spatial.distance.cdist(geo_df, centroids,"euclidean")

# 가장 거리가 짧은 중심점의 cluster로 할당
cluster_num = np.argmin(distance, axis=1)

# 결과 확인
result = geo_df.copy()
result["cluster"] = np.array(cluster_num)
result.head()

In [ ]:
plt.figure(figsize=(10,5))
sns.scatterplot(x="longitude", y="latitude", hue="cluster", data=result,palette="Paired");

In [ ]:
centroids_2 = result.groupby("cluster").mean()
centroids_2

In [ ]:
plt.figure(figsize=(10,5))
sns.scatterplot(x="longitude", y="latitude", hue="cluster", data=result,palette="Paired");

In [ ]:
from sklearn.cluster import KMeans

kcls=KMeans(n_clusters=9)
cst_group=kcls.fit_predict(geo_df)

print(cst_group)

In [ ]:
for i in range(9):
    labels=geo_df[cst_group==i]
    plt.scatter(labels['longitude'],labels['latitude'],label=i)

plt.legend()
plt.xlabel('longitude')
plt.ylabel('latitude')
plt.show()

In [ ]:
input_data['geo_label']=cst_group

In [ ]:
input_data.head(10)

In [ ]:
print(input_data['geo_label'].value_counts())

# Data split

In [ ]:
input_data_y = input_data['average_sale_price'].copy()
input_data_X = input_data.drop(['average_sale_price'], axis=1)

In [ ]:
from sklearn.model_selection import train_test_split

tr_val_X, test_X, tr_val_y, test_y = train_test_split(
    input_data_X, 
    input_data_y, 
    test_size = 0.2,      
    shuffle=True,         
    random_state=42)      

train_X, valid_X, train_y, valid_y = train_test_split(
    tr_val_X, 
    tr_val_y, 
    test_size = 0.2,      
    shuffle=True,         
    random_state=42)   

# Modeling

- XGB
- LGBM

In [ ]:
from sklearn.model_selection import train_test_split, cross_val_score, KFold, TimeSeriesSplit,GridSearchCV
from sklearn.metrics import mean_squared_error

from lightgbm import LGBMRegressor
from lightgbm import plot_importance 
from xgboost import XGBRegressor
from xgboost import plot_importance

from keras.callbacks import EarlyStopping, ModelCheckpoint

In [ ]:
model_xgb = XGBRegressor(n_estimators=500, learning_rate=0.5, max_depth=3)
#model_xgb = XGBRegressor(n_estimaotrs=100, learning_rate=1.0, max_depth=3, early_stopping_rounds=2)
# 학습
model_xgb.fit(train_X, train_y, early_stopping_rounds=100, eval_set=[(valid_X, valid_y)],verbose=True)

In [ ]:
##XGB
# pred_xgb = model_xgb.predict(test_X)
# # 학습후처리


# rescaled_actual = scaler2.inverse_transform(scaler2_df.values.reshape(-1,1))
# rescaled_pred = scaler2.inverse_transform(pred_xgb.reshape(-1,1))

In [ ]:
model_lgbm = LGBMRegressor(n_estimators=500, learning_rate=0.05)

model_lgbm.fit(train_X, train_y, early_stopping_rounds=100, eval_set=[(valid_X, valid_y)], verbose=True)  
pred_lgbm = model_lgbm.predict(test_X)

# Prediction

In [ ]:
def plot_history(history):
    #hist = pd.DataFrame(history.history)
    #history['epoch'] = history.epoch
    
    plt.figure(figsize=(8,12))
    
    # Mean Abs Error : 평균 절대 오차, 측정값에서 오차의 크기로 측정값과 실제값과의 차이, 절대 오차의 평균  
    # -> 측정하고자 하는 값을 정확하게 측정하지 못함으로써 발생
    plt.subplot(2,1,1)
    plt.xlabel('Epoch')
    plt.ylabel('Mean Abs Error')
    plt.plot(hist['epoch'], hist['mae'],
           label='Train Error')
    plt.plot(hist['epoch'], hist['val_mae'],
           label = 'Val Error')
    plt.legend()
    
    # Mean Square Error : 평균 제곱 오차, 오차의 제복에 대한 평균을 취한 값
    plt.subplot(2,1,2)
    plt.xlabel('Epoch')
    plt.ylabel('Mean Square Error')
    plt.plot(hist['epoch'], hist['mse'],
           label='Train Error')
    plt.plot(hist['epoch'], hist['val_mse'],
           label = 'Val Error')
    plt.legend()
    plt.show()
    
def show_pred(true_y, pred) :
    #true_y = true_y.to_numpy()
    true_y = np.ravel(true_y)
    pred = np.array(pred.reshape(-1,1))
    
    df_result = pd.DataFrame(list(zip(true_y, pred)), columns=['true_y', 'prediction'])
    return df_result

def show_mse_rmse(test_y, pred) :
    mse = mean_squared_error(test_y, pred)
    print("mse : %f" % mse)
    
    rmse = np.sqrt(mse)
    print("rmse: %f \n" %rmse)
    
def show_prediction_error(test_y, pred) :
    true_y = test_y.to_numpy()
    true_y = np.ravel(true_y)
    error = pred - true_y
    plt.hist(error, bins=25)
    plt.xlabel("Prediction Error")
    _ = plt.ylabel("Count")
    
def feature_importance(model_xgb) : 
    %matplotlib inline
    plt.rcParams['axes.unicode_minus'] = False
    font_path = "C:/Windows/Fonts/NGULIM.TTF"
    font = fm.FontProperties(fname=font_path).get_name()
    rc('font', family=font)

    fig, ax = plt.subplots(figsize=(10,12))
    plot_importance(model_xgb, ax=ax)
    
def graph(pred, test_label) :
    plt.figure(figsize=(16, 9))
    plt.plot(test_label, label = 'actual')
    plt.plot(pred, label = 'prediction')
    plt.legend()
    plt.show()

In [ ]:
#show_pred(test_y, pred_xgb)

In [ ]:
show_pred(test_y, pred_lgbm)

In [ ]:
#show_mse_rmse(test_y, pred_xgb)

In [ ]:
show_mse_rmse(test_y,pred_lgbm)

In [ ]:
#show_prediction_error(test_y,pred_xgb)